<a href="https://githubtocolab.com/Eunseob/purdue_me597/blob/main/lab/lab10/L10_Colab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 10.3 Machine Learning 3 - Real-time Sound Recognition for Classification

## Entire remote monitoring system


## 3.1 MTConnect for pump monitoring 

In the prvious section, we performed utilizing the CNN model by TensorFlow Lite to predict the running state of the vacuum pump. Let's make an entire monitoring system as [Figure 1](https://colab.research.google.com/github/Eunseob/purdue_me597/blob/main/lab/lab10/L10_Colab1.ipynb#scrollTo=x_ZERkf6r_E5). Depending on the project or system requirements, you skip the MTConnect part. However, considering the potential use of MTConnect to collect machine data in the future, design and create an MTConnect agent to collect and merge the running state prediction (Event) and sound level (Sample) of the vacuum pump.

The idea is that we know both the execution state (OFF/ON, Event) and the vacuum state (Vacuuming/Air leaking, Event) from the CNN model. Also, we calculate the sound level (Sample). The schematic and data items are shown in Figure 7.

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab10_fig7.png?raw=true" width="80%">

*Figure 7 Schematic and data items of MTConnect*

The examples of 'agent.cfg' and 'device.xml' are below. You may have different an XML data structure or data item definitions. Also, you can check the example of MTConnect agent running on the server computer: http://mepotr16.ecn.purdue.edu:5000/. Recall the previous labs to run MTConnect service on Raspberry Pi and reuse the agent execution file and other directory/files (schema, style, and so on).

<br>

---
**<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/icon_conf.png?raw=tru" width="20"> agent.cfg**

```
Devices = device.xml
AllowPut = true
Port = 5000
ReconnectInterval = 1000
BufferSize = 17
SchemaVersion = 1.4
MonitorConfigFiles = true
Pretty = true
# MinimumConfigReloadAge = 30

Adapters {
   # Log file has all machines with device name prefixed
   Device {
      Device = vacuum_pump
      Host = 127.0.0.1
      Port = 7878
   }
}

Files {
    schemas {
        Path = ./schemas
        Location = /schemas/
    }
    styles {
        Path = ./styles
        Location = /styles/
    }
    Favicon {
        Path = ./styles/favicon.ico
        Location = /favicon.ico
    }
}

DevicesStyle { Location = /styles/Devices.xsl }
StreamsStyle { Location = /styles/Streams.xsl }

# Logger Configuration
logger_config
{
    logging_level = debug
}
```
---

<br>

---
**<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/icon_xml.png?raw=tru" width="20"> device.xml**
```
<?xml version="1.0" encoding="UTF-8"?>
<MTConnectDevices xmlns:m="urn:mtconnect.org:MTConnectDevices:1.1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="urn:mtconnect.org:MTConnectDevices:1.1" xsi:schemaLocation="urn:mtconnect.org:MTConnectDevices:1.1 http://www.mtconnect.org/schemas/MTConnectDevices_1.1.xsd">
	<Header creationTime="2010-03-04T18:44:40+00:00" sender="localhost" instanceId="1267728234" bufferSize="131072" version="1.1"/>
	<Devices>
		<Device id="vacuum_pump" name="vacuum_pump" uuid="001">
			<DataItems>
				<DataItem category="EVENT" id="001" name="avail" type="AVAILABILITY"/>
				<DataItem category="EVENT" id="e1" name="execution" type="EXECUTION"/>
				<DataItem category="EVENT" id="vs1" name="vacuum_state" type="STATE"/>
				<DataItem category="SAMPLE" id="spl" name="sound_level" type="SOUND_LEVEL" units="DECIBEL"/>
				
			</DataItems>
		</Device>
	</Devices>
</MTConnectDevices>
```

---

The MTConnect adapter sample code ([lab10_sample_adapter.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab10/lab10_sample_code/lab10_sample_adapter.py?raw=true)) is also given. The interpretation algorithm to take the CNN model output as MTConnect data item is shown in Figure 8. If the output is 0, execution is "OFF" and vacuum state is "N/A" (not available). If the output is 1, execution is "ON" and vacuum state "Air leaking". Lastly, if the output is 2, execution is "ON" and vacuum state "Vacuuming". However, it is recommended to try programming the adapter yourself. If you use the given sample code, you have to modify some lines. The incomplete part of the sample script is below. You have to change the variables which is wrarpped by aserteric marks (*). By completing the sample code , perform Task 3.1.

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab10_fig8.png?raw=true" width="70%">

*Figure 8 Algorithm to take MTConnect data item from CNN model output*

Note that your CNN model (h5 format), [mtconnect_adapter.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab5/adapter_sample/mtconnect_adapter.py?raw=true), and [data_item.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab5/adapter_sample/data_item.py?raw=true) must be in the same directory with the adapter.






---

**<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/icon_Python.png?raw=tru" width="20">Python - Python3 (Part of [lab10_sample_adapter.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab10/lab10_sample_code/lab10_sample_adapter.py?raw=true))**

```
## == GLOBAL CONSTANT ==
SAMPLE = "sample" # sample string for MTConnect HTML sample method
CURRENT = "current" # current string for MTConnect HTML current method
SAMP_RATE = int(??) # sampling rate
*SAMP_RATE* = int(?) # sampling rate
*CHUNK* = int(?) # chunk size
*AGENT* = "http://?ip?:?port?/" # MTConnect agent ip:port

*N* = int(?) # number of sequence to take sound
*N_FFT* = int(?) # number of FFT
*N_MELS* = int(?) # number of Mel filter bank

*model_file* = '?20230309_212154_Prelab10_CNN_model1.h5?' # CNN model fileanme. Note that this file must be in the same directory

```

---

### Task 3.1

Run MTConnect agent and adapter on Raspberry Pi with given informationa and sample. Capture '/current' response of the MTConnect agent as Figure 9 from **your laptop** and attach it to the report. While performing this, check if your adapter is working correctly. You see the terminal as Figure 10. 



---

Place your screenshot here.

---



<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab10_fig9.png?raw=true" width="80%">

*Figure 9 MTConnect agent '/current' response*

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab10_fig10.png?raw=true" width="50%">

*Figure 10 Capture of terminal in MTConnect adapter is running*

## 3.2 MySQL database and Grafana dashboard

Let's move on to the data aggregation for MySQL database and visualization by Grafana. To collect MTConnect agent data, use the sample Python script ([mtconnect_collecor.py](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab7/mtconnect_collector.py?raw=true)).

MySQL and Grafana information is the same as the previous lab. However they are summarized again below.

<br>

*   <img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/icon_MySQL.png?raw=tru" width="35"> **MySQL**
  *   DNS: mepotrb16.ecn.purdue.edu
  *   Port: 3306
  *   Account: yourname (e.g., johndoe)
  *   Database (schema): ME597Spring23
  *   Table: yourname_lab6 (e.g., johndoe_lab6)
      * Reuse the table you created in Lab 6.

*   <img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/icon_Grafana.png?raw=tru" width="20"> **Grafana**
  *   DNS: mepotrb16.ecn.purdue.edu
  *   Port: 3000
  *   Grafana web page URL: http://mepotrb16.ecn.ecn.purdue.edu:3000/
  *   Account (username): yourname (e.g., johndoe) OR your purdue email account



<br>


The sample Grafana dashboard is 'Lab10 sample' in '0_SAMPLE' dashboard folder as Figure 11. Perform Task 3.2 to conclude this lab.

<br>

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab10_fig11.png?raw=true" width="100%">

*Figure 11 Lab10 Grafana dashboard sample*


### Task 3.2

Create a dashboard as Figure 11 in your folder created in Lab 7 and save it as the dashboard name of 'lab10 your name', e.g., 'lab10 john doe'. The requirements of the dashboard are below.



*   Discrete panel for Execution
*   Discrete panel for Vacuum state
*   Time series panel for Sound level
*   (Optional) Add more if you want



Capture the dashboard and attach it to the report.

After creating the dashboard, confirm if the entire monitoring system works in real time by refreshing the dashboard. TA will change the operational state of the pump during lab occasionally.



---

Place your screenshot here.

---



## Task 3.3 Further discussion

Probably, sometimes you may see false results as Figure 12. In this example, although all actual state (red circle in Figure 12) was Air leaking, sporadically the inference state might be Vacuuming or vice versa. Discuss how to fix this problem in any aspect.
For instance, we can discuss various aspects such as machine learning/deep learning architecture selection, training, algorithm, and so on.

<br>

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab10_fig12.png?raw=true" width="100%">

*Figure 12 False prediction result*

<br>




---

Place your answer to Task 3.3 here.

---



## Lab10 Summary and Deliverables

Answer the following questions for your achievements




### Q1. Please summarize Lab10.

---

Write down A1 here.

---


### Q2. What skills did you have to develop to accomplish this project?

---

Wirte down A2 here.

---

### Q3. What aspects of this project were the most beneficial for your learning?

---

Wirte down A3 here.

---

### Q4. What challenges did you encounter in completing the project?

---

Wirte down A4 here.

---

### Q5. How did you overcome the challenges or remedy the problems encountered?

---

Wirte down A5 here.

---

<br></br>

Get back to [Lab Index Page](https://colab.research.google.com/github/Eunseob/purdue_me597/blob/main/index.ipynb)